In [ ]:
## 下載爬蟲、截圖、裁圖、資料庫相關套件

!pip install selenium
!pip install Pillow
!pip install pytesseract 
!pip install pymysql 

In [18]:
## 用selenium截圖

from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = Chrome('C:/Users/user/Desktop/chromedriver')
url = 'https://ntc.im/pn/' 
driver.get(url)
time.sleep(8)

# 找到目標的位置,截圖並存檔
action = ActionChains(driver)
final_path = "./ScreenShot.png"
driver.find_element_by_xpath('//*[@id="intro"]/div/div[1]/div/h2').screenshot(final_path)

driver.close()

C:\Users\user\AppData\Local\Temp/ipykernel_4744/3954492159.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('C:/Users/user/Desktop/chromedriver')
C:\Users\user\AppData\Local\Temp/ipykernel_4744/3954492159.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="intro"]/div/div[1]/div/h2').screenshot(final_path)


In [19]:
## 較大範圍的截圖可事後裁切出指定位置

from PIL import Image

img = Image.open("./ScreenShot.png")
img2 = img.crop((240,240,780,360))  # 裁切大小
img2.save("./ScreenShot2.png")  # 儲存圖片

In [20]:
## 將截圖的文字擷取下來轉成str

import pytesseract    # 需要下載執行檔並將檔案路徑新增至環境變數   # 安裝時語言可多勾選繁體中文

P = Image.open(final_path)
STR = pytesseract.image_to_string(P,lang='chi_tra').split('\n')[0].strip()  # 因為文字都是中文,用'chi_tra'可翻譯得更正確
print(STR)

你不需要很厲害才能開始


In [21]:
## 連線至MySQL,新增一個table以存取文字和圖片

import pymysql

def db_init():
   db = pymysql.connect(
       host='127.0.0.1',
       user='root',
       password='0710',
       port=3306,    
       db='newschema' 
   )
   cursor = db.cursor(pymysql.cursors.DictCursor)
   return db, cursor

db, cursor = db_init()

# 先新增table,欄位包含ID、char和image_path
sql = "CREATE TABLE IF NOT EXISTS `newschema`.`text` (`id` INT NOT NULL AUTO_INCREMENT,`char` VARCHAR(45) NULL,`image_path` VARCHAR(100) NULL,PRIMARY KEY (`id`));"
cursor.execute(sql) 
db.commit()

# 將字串和圖檔路徑存入MySQL Database
sql2 = f"INSERT INTO `newschema`.`text` (`char`,`image_path`) VALUE ('{STR}','{final_path}');"   
cursor.execute(sql2) 
db.commit()

db.close

<bound method Connection.close of <pymysql.connections.Connection object at 0x000000D540A6B8B0>>

In [25]:
## 從MySQL Database搜尋並取用字串和圖檔路徑

db, cursor = db_init()
sql3 = "SELECT `char`,`image_path` FROM `newschema`.`text` where id = 1;"   
cursor.execute(sql3) 
c = cursor.fetchone()
char = str(c['char'])
image_path = c['image_path']
db.close
print(char)

你不需要很厲害才能開始 ./ScreenShot.png


In [29]:
## 將搜尋出來的文字和圖檔上傳至網頁

driver = Chrome('C:/Users/user/Desktop/chromedriver')
url2 = 'https://ntc.microtrend.tw/202202?classid=DV101&studentid=05'
driver.get(url2)
time.sleep(5)

# 將文字填入
driver.execute_script('document.getElementsByClassName("align-self-center me-2 ")[1].insertAdjacentElement("afterend",document.createElement("input"))')   # 在網站的指定位置新增一個input欄位
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("autocomplete","false")')   # 設置input欄位的Attribute和value
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("type","text")')
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("class","form-control border-0 border-bottom")')
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("name","anwser")')
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("id","anwser")')
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("placeholder","放入OCR解析的字串")')
driver.execute_script(f'document.getElementsByTagName("input")[4].setAttribute("value","{char}")')   # 在value填入文字
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("required","required")')
driver.execute_script('document.getElementsByTagName("input")[4].setAttribute("readonly","")')
#driver.execute_script(f'document.getElementById("anwser").value="{char}"')

# 將圖片上傳
pic_upload = driver.find_element_by_xpath('//*[@id="img_src"]')
pic_upload.send_keys(image_path)  


# 按下提交按鈕
submit_button = driver.find_element_by_class_name('btn')
submit_button.click()

C:\Users\user\AppData\Local\Temp/ipykernel_4744/1657512496.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('C:/Users/user/Desktop/chromedriver')
C:\Users\user\AppData\Local\Temp/ipykernel_4744/1657512496.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pic_upload = driver.find_element_by_xpath('//*[@id="img_src"]')
C:\Users\user\AppData\Local\Temp/ipykernel_4744/1657512496.py:27: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  submit_button = driver.find_element_by_class_name('btn')
